<a target="_blank" href="https://colab.research.google.com/github/pjyi2147/CUDA_HTN_Workshop/blob/main/notebook.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 0. Setup

## Select GPU for CUDA

Select `Runtime -> Change Runtime Type -> Select Python and T4 GPU`

Press Connect T4 on top right of the notebook.

Run the following snippets:


In [ ]:
!nvcc --version
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

In [ ]:
%%cuda
#include <stdio.h>

__global__ void hello() {
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main() {
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

You should get something similar to this:

![Jupyter result](https://github.com/user-attachments/assets/eff2ae68-0b50-4e8e-8e73-26ccc5563754)

# 2. Hello World!





In [ ]:
%%cuda

#include <stdio.h>

// Host (CPU) Hello World function
void helloWorld()
{
    printf("Hello world!\n");
}


// Device (GPU) Hello World kernel
__global__ void helloWorldGPU()
{
    printf("Hello world from GPU!\n");
}

int main() {
    helloWorld();
    helloWorldGPU<<<1,1>>>();
    // Try commenting out this line
    // Do you see GPU output? What does it mean?
    cudaDeviceSynchronize();
    return 0;
}

# 3. Vector Addition


## 3.1 Single Integer Addition

In [ ]:
%%cuda

#include <stdio.h>
__global__ void add(int *a, int *b, int *c)
{
    *c = *a + *b;
}

int main()
{
    int a = 3, b = 2, c = 0;    // host copies of a, b, c
    int *d_a, *d_b, *d_c;       // device copies of a, b, c
    int size = sizeof(int);

    //// Allocate memory to device
    //// Allocate memory to d_a, d_b, and d_c using the following example
    /* YOUR CODE HERE */
    // cudaMalloc((void **)&d_a, size);


    //// Copy inputs to device
    //// Copy input to d_a, d_b using the following example
    /* YOUR CODE HERE */
    // cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);


    //// Launch kernel
    add<<<1, 1>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    //// Copy result from device (d_c) to host (c)
    //// What is the difference of this line compared to above cudaMemcpy?
    /* YOUR CODE HERE */
    // cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

    //// print result, is result 5 instead of 0?
    printf("%d\n", c);

    //// Memory cleanup!
    //// Clean up memory for d_a, d_b, d_c with following example
    /* YOUR CODE HERE */
    // cudaFree(d_a);

    return 0;
}


## 3.2 Vector Addition with Thread Blocks

In [ ]:
%%cuda

#include <stdio.h>

#define N 16

void random_ints(int *a, int n)
{
    for (int i = 0; i < n; i++)
    {
        a[i] = rand() % 10;
    }
}

__global__ void add(int *a, int *b, int *c)
{
    //// We are using the same number of blocks as the array length
    c[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x];
}

int main()
{
    int size = sizeof(int) * N;
    int *a = (int *)malloc(size);
    random_ints(a, N);
    int *b = (int *)malloc(size);
    random_ints(b, N);
    int *c = (int *)malloc(size);
    memset(c, 0, size);

    // device copies of a, b, c
    int *d_a, *d_b, *d_c;

    //// Allocate memory to d_a, d_b, and d_c
    /* YOUR CODE HERE */

    //// Copy input to d_a, d_b
    /* YOUR CODE HERE */

    //// Launch kernel
    //// N is placed in left side of the brackets (blocks)
    add<<<N, 1>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    //// Copy result from device (d_c) to host (c)
    /* YOUR CODE HERE */

    //// print result, is result correct?
    for (int i = 0; i < N; i++)
    {
        printf("i = %d, %d + %d = %d\n", i, a[i], b[i], c[i]);
    }

    //// Memory cleanup!
    //// Clean up memory for d_a, d_b, d_c
    /* YOUR CODE HERE */

    //// Of course the same for host memory, too.
    free(a);
    free(b);
    free(c);

    return 0;
}


## 3.3 Vector Addition with Threads

In [ ]:
%%cuda

#include <stdio.h>

#define N 16

void random_ints(int *a, int n)
{
    for (int i = 0; i < n; i++)
    {
        a[i] = rand() % 10;
    }
}

__global__ void add(int *a, int *b, int *c)
{
    //// We are using the same number of threads as the array length
    c[threadIdx.x] = a[threadIdx.x] + b[threadIdx.x];
}

int main()
{
    int size = sizeof(int) * N;
    int *a = (int *)malloc(size);
    random_ints(a, N);
    int *b = (int *)malloc(size);
    random_ints(b, N);
    int *c = (int *)malloc(size);
    memset(c, 0, size);

    //// device copies of a, b, c
    int *d_a, *d_b, *d_c;

    //// Allocate memory to d_a, d_b, and d_c
    /* YOUR CODE HERE */

    //// Copy input to d_a, d_b
    /* YOUR CODE HERE */

    //// Launch kernel
    //// N is placed in right side of the brackets (threads)
    add<<<1, N>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    //// Copy result from device (d_c) to host (c)
    /* YOUR CODE HERE */


    //// print result, is result correct?
    for (int i = 0; i < N; i++)
    {
        printf("i = %d, %d + %d = %d\n", i, a[i], b[i], c[i]);
    }

    //// Memory cleanup!
    //// Clean up memory for d_a, d_b, d_c
    /* YOUR CODE HERE */

    //// Of course, we need to do the same for host memory, too.
    free(a);
    free(b);
    free(c);

    return 0;
}


## 3.4 Vector Addition with Thread Blocks and Threads (Grids)

In [ ]:
%%cuda

#include <stdio.h>

#define N 32

#define BLOCK_SIZE 16

void random_ints(int *a, int n)
{
    for (int i = 0; i < n; i++)
    {
        a[i] = rand() % 10;
    }
}

__global__ void add(int *a, int *b, int *c)
{
    // We now use both blockIdx and threadIdx!
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    c[idx] = a[idx] + b[idx];
}

int main()
{
    int size = sizeof(int) * N;
    int *a = (int *)malloc(size);
    random_ints(a, N);
    int *b = (int *)malloc(size);
    random_ints(b, N);
    int *c = (int *)malloc(size);
    memset(c, 0, size);

    // device copies of a, b, c
    int *d_a, *d_b, *d_c;

    //// Allocate memory to d_a, d_b, and d_c
    /* YOUR CODE HERE */

    //// Copy input to d_a, d_b
    /* YOUR CODE HERE */

    //// Launch kernel
    int NUMBLOCKS = N / BLOCK_SIZE;
    add<<<NUMBLOCKS, BLOCK_SIZE>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    //// Copy result from device (d_c) to host (c)
    /* YOUR CODE HERE */

    //// print result, is result correct?
    for (int i = 0; i < N; i++)
    {
        printf("i = %d, %d + %d = %d\n", i, a[i], b[i], c[i]);
    }

    //// Memory cleanup!
    //// Clean up memory for d_a, d_b, d_c
    /* YOUR CODE HERE */


    //// Of course the same for host memory, too.
    free(a);
    free(b);
    free(c);

    return 0;
}


## 3.5 Arbitrary Length Vector Addition

We have been using array lengths divisible by `BLOCK_SIZE`. What can happen if we have lists of arbitrary length? 

How can we support lists of arbitrary length with fixed `BLOCK_SIZE`?

In [ ]:
%%cuda

#include <stdio.h>

// Arbitrary array length
#define N 50

#define BLOCK_SIZE 16

void random_ints(int *a, int n)
{
    for (int i = 0; i < n; i++)
    {
        a[i] = rand() % 10;
    }
}

// Why length variable is added to the kernel now?
__global__ void add(int *a, int *b, int *c, int len)
{
    // check the difference here
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < len)
    {
        c[idx] = a[idx] + b[idx];
    }
}

int main()
{
    int size = sizeof(int) * N;
    int *a = (int *)malloc(size);
    random_ints(a, N);
    int *b = (int *)malloc(size);
    random_ints(b, N);
    int *c = (int *)malloc(size);
    memset(c, 0, size);

    // device copies of a, b, c
    int *d_a, *d_b, *d_c;

    //// Allocate memory to d_a, d_b, and d_c
    /* YOUR CODE HERE */

    //// Copy input to d_b using the following example
    /* YOUR CODE HERE */

    //// Launch kernel
    //// what's the difference here?
    int NUMBLOCKS = (N - 1) / BLOCK_SIZE + 1;
    add<<<NUMBLOCKS, BLOCK_SIZE>>>(d_a, d_b, d_c, N);
    cudaDeviceSynchronize();

    //// Copy result from device (d_c) to host (c)
    /* YOUR CODE HERE */

    //// print result, is result correct?
    for (int i = 0; i < N; i++)
    {
        printf("i = %d, %d + %d = %d\n", i, a[i], b[i], c[i]);
    }

    //// Memory cleanup!
    //// Clean up memory for d_a, d_b, d_c
    /* YOUR CODE HERE */

    //// Of course the same for host memory, too.
    free(a);
    free(b);
    free(c);

    return 0;
}


## 3.6 Matrix Addition

Use `dim3` structure to use (x, y, z) coordinates in thread blocks.

In [ ]:
%%cuda

#include <stdio.h>

// 7 x 7 Matrix
#define N_X 7
#define N_Y 7

// 4 x 4 blocksize
#define BLOCK_SIZE_X 4
#define BLOCK_SIZE_Y 4

void random_ints(int *a, int n)
{
    for (int i = 0; i < n; i++)
    {
        a[i] = rand() % 10;
    }
}

__global__ void add(int *a, int *b, int *c, int len_x, int len_y)
{
    // Calculate yidx and xidx yourself. Use blockIdx, blockDim, and threadIdx
    /* YOUR CODE HERE */
    int yidx = ;
    int xidx = ;

    // What needs to be checked here?
    if ()
    {
        // What is the index we should use here?
        c[] = a[] + b[];
    }
}

int main()
{
    int size = sizeof(int) * N_X * N_Y;
    int *a = (int *)malloc(size);
    random_ints(a, N_X * N_Y);
    int *b = (int *)malloc(size);
    random_ints(b, N_X * N_Y);
    int *c = (int *)malloc(size);
    memset(c, 0, size);

    // device copies of a, b, c
    int *d_a, *d_b, *d_c;

    //// Allocate memory to d_a, d_b and d_c
    /* YOUR CODE HERE */


    //// Copy input to d_a, d_b
    /* YOUR CODE HERE */


    //// Launch kernel
    //// what is the difference here? We use dim3 struct (x, y, z)
    dim3 NUMBLOCKS = {(N_X - 1) / BLOCK_SIZE_X + 1, (N_Y - 1) / BLOCK_SIZE_Y + 1 ,1};
    add<<<NUMBLOCKS, {BLOCK_SIZE_X, BLOCK_SIZE_Y, 1}>>>(d_a, d_b, d_c, N_X, N_Y);
    cudaDeviceSynchronize();

    //// Copy result from device (d_c) to host (c)
    /* YOUR CODE HERE */



    //// print result, is result correct?
    for (int i = 0; i < N_Y; i++)
    {
        for (int j = 0; j < N_X; j++)
        {
            printf("i = %d, j = %d, %d + %d = %d\n", i, j, a[i*N_X + j], b[i*N_X + j], c[i*N_X + j]);
        }
    }

    //// Memory cleanup!
    //// Clean up memory for d_a, d_b, d_c


    //// Of course the same for host memory, too.
    free(a);
    free(b);
    free(c);

    return 0;
}


# 3.7 Benchmark for 400M vector subtraction

We subtract `a` from `a` for an easier validation of calculation, all elements in `c` must be 0 after calculation!

Why GPU time takes longer than CPU computation meanwhile GPU computation itself takes way shorter than CPU?

In [ ]:
%%cuda

#include <assert.h>
#include <stdio.h>
#include <sys/time.h>

#define N 400000000     // 400M elements

#define BLOCK_SIZE 1024

void random_ints(int *a, int n)
{
    for (int i = 0; i < n; i++)
    {
        a[i] = rand() % 10;
    }
}

__global__ void sub(int *a, int *c, int len)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < len)
    {
        c[idx] = a[idx] - a[idx];
    }
}

void sub_cpu(int *a, int *c, int len)
{
    for (int i = 0; i < len; i++)
    {
        c[i] = a[i] - a[i];
    }
}

void gpu(int *a, int *c, int len)
{
    size_t size = sizeof(int) * N;
    // device copies of a, c
    int *d_a, *d_c;

    // Allocate memory to device
    // Allocate memory to d_b and d_c using the following example
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_c, size);

    //// Copy inputs to device
    //// Copy input to d_b using the following example
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);

    //// Launch kernel
    //// what's the difference here?
    int NUMBLOCKS = (N - 1) / BLOCK_SIZE + 1;

    struct timeval gpustart, gpuend;
    gettimeofday(&gpustart, NULL);
    sub<<<NUMBLOCKS, BLOCK_SIZE>>>(d_a, d_c, N);
    cudaDeviceSynchronize();
    gettimeofday(&gpuend, NULL);

    double secs = (double)(gpuend.tv_usec - gpustart.tv_usec) / 1000000 + (double)(gpuend.tv_sec - gpustart.tv_sec);
    printf("gpu computation time taken %f s\n", secs);

    //// Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    //// Clean up memory for d_a, d_c
    cudaFree(d_a);
    cudaFree(d_c);
}

int main()
{
    size_t size = sizeof(int) * N;
    int *a = (int *)malloc(size);
    random_ints(a, N);
    int *c = (int *)malloc(size);
    random_ints(c, N);

    // benchmark CPU (one thread)
    struct timeval cpustart, cpuend;
    gettimeofday(&cpustart, NULL);
    sub_cpu(a, c, N);
    gettimeofday(&cpuend, NULL);

    for (int i = 0; i < N; i++)
    {
        assert(c[i] == 0);
    }

    double secs = (double)(cpuend.tv_usec - cpustart.tv_usec) / 1000000 + (double)(cpuend.tv_sec - cpustart.tv_sec);
    printf("cpu computation time taken %f s\n", secs);

    // reset c with random values
    random_ints(c, N);

    // benchmark GPU
    struct timeval gpustart, gpuend;
    gettimeofday(&gpustart, NULL);
    gpu(a, c, N);
    gettimeofday(&gpuend, NULL);

    for (int i = 0; i < N; i++)
    {
        assert(c[i] == 0);
    }

    secs = (double)(gpuend.tv_usec - gpustart.tv_usec) / 1000000 + (double)(gpuend.tv_sec - gpustart.tv_sec);
    printf("gpu time taken %f s\n", secs);

    //// Memory cleanup!
    free(a);
    free(c);

    return 0;
}


## 3.8 Submit benchmarks of subtraction 50000 x 50000 array!

Using above benchmark code, try to write your own 2D matrix subtraction kernel with benchmark results and send me a PR to this repo!

You can write your own code in any way (do not need to be this notebook!) you would like, but it needs to be CPU vs GPU with different configurations of `BLOCK_SIZE`, array size, etc.

The instructions are in `README.md` file!

# 4. Next Steps

## Parallel Algorithm Example 1
## 4.1 1D reduction (1D max, min, sum, ...)

## 4.2 Resources:

Books: Programming Massively Parallel Processors 4th Edition 

![PMPP](https://github.com/user-attachments/assets/01cf0739-d720-4ec4-9c83-412c7ac9d824)

LLM Specific: [llm.c](https://github.com/karpathy/llm.c) by Andrej karpathy

## 4.3 Answers:

Notebook with answers with cuda code can be found in `answer` branch of the repository.